## Hyperparameter evaluation

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pandas as pd
import numpy as np
import quiche as qu
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import scanpy as sc
import mudata
import logging
import matplotlib.cm as cm
import imageio as io

%reload_ext autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

### simulate data

In [ ]:
# compartment_colormap = pd.DataFrame({'mask_name': ['cancer_core', 'cancer_border', 'stroma', 'immune1', np.nan], 
#                                      'color': ['blue', 'deepskyblue','firebrick', 'orange', 'black']})

# compartment_colormap = pd.DataFrame({'mask_name': ['cancer_core', 'cancer_border', 'stroma', 'immune1', np.nan], 
#                                      'color': ['#1885F2', '#78CE8B','#B46CDA', '#D78F09', 'black']})

# compartment_colormap = pd.DataFrame({'mask_name': ['cancer_core', 'cancer_border', 'stroma', 'immune1', np.nan], 
#                                      'color': ['blue', 'deepskyblue','#A5243D', 'orange', 'black']})


# compartmentalized_tumors = ['TMA31_R3C1', 'TMA31_R3C9', 'TMA41_R4C4', 'TMA31_R4C5', 'TMA31_R5C4', 'TMA31_R5C5', 'TMA31_R7C1', 'TMA32_R5C7', 'TMA32_R8C5', 'TMA32_R10C4', 'TMA33_R5C8',
#                             'TMA33_R8C4', 'TMA33_R9C4', 'TMA33_R10C5', 'TMA33_R12C2', 'TMA34_R4C2', 'TMA34_R9C8', 'TMA34_R12C3', 'TMA35_R3C2', 'TMA35_R4C3', 'TMA36_R2C7',
#                             'TMA36_R9C9', 'TMA37_R3C1', 'TMA37_R4C4', 'TMA37_R4C7', 'TMA37_R7C4', 'TMA37_R10C5', 'TMA38_R5C2', 'TMA39_R5C6', 'TMA39_R1C1', 'TMA39_R2C4', 'TMA39_R3C4',
#                             'TMA39_R5C4', 'TMA39_R5C6', 'TMA39_R5C8', 'TMA39_R6C1', 'TMA39_R9C2', 'TMA39_R9C6', 'TMA40_R4C7', 'TMA40_R5C2', 'TMA40_R6C3', 'TMA40_R6C6', 'TMA40_R7C6',
#                             'TMA40_R7C7', 'TMA40_R8C6', 'TMA40_R10C7', 'TMA41_R1C3', 'TMA41_R2C3', 'TMA41_R4C2', 'TMA41_R4C3', 'TMA41_R4C4', 'TMA42_R2C2', 'TMA42_R3C5', 'TMA42_R4C1',
#                             'TMA42_R6C1', 'TMA42_R6C5', 'TMA42_R7C4', 'TMA43_R1C3', 'TMA43_R3C3', 'TMA43_R5C7', 'TMA43_R8C7', 'TMA43_R9C8', 'TMA43_R11C5', 'TMA44_R3C3', 'TMA44_R3C7',
#                             'TMA44_R7C2', 'TMA44_R7C6', 'TMA44_R8C1', 'TMA44_R8C3', 'TMA44_R9C5', 'TMA44_R10C6', 'TMA44_R12C2', 'TMA44_R12C7', 'TMA44_R13C7', 'TMA44_R14C7']

# base_dir = '/Volumes/Shared/Noah Greenwald/TNBC_Cohorts/SPAIN/intermediate_files'
# cell_table = pd.read_csv(os.path.join(base_dir, 'post_processing', 'cell_table_size_normalized_samples_cell_labels_updated.csv'))

# annotations_by_mask = pd.read_csv(os.path.join(base_dir, 'mask_dir', 'individual_masks-no_tagg_tls', 'cell_annotation_mask.csv'))
# annotations_by_mask_merged = pd.merge(annotations_by_mask, cell_table.loc[:, ['fov', 'label', 'centroid-1', 'centroid-0']], on = ['fov', 'label'])
# annotations_by_mask_merged['mask_name'].replace(['stroma_core', 'stroma_border'], ['stroma', 'stroma'], inplace = True)
# annotations_by_mask_merged = annotations_by_mask_merged[np.isin(annotations_by_mask_merged.fov, compartmentalized_tumors)]

# adata_expression = anndata.read_h5ad(os.path.join('data', 'simulated', 'adata_simulated_expression_groups_large.h5ad'))

# num_to_change = 50
# count_df = annotations_by_mask_merged.groupby(['fov', 'mask_name']).count()['label'].unstack()
# ratio_df = pd.Series([800, 1200, 1600], index = ['cancer_border', 'cancer_core', 'stroma'])
# fov_list = (count_df > ratio_df + num_to_change).sum(axis = 1)
# fov_list = list(fov_list[fov_list == 3].index)
# ratio_df = pd.Series([800, 1200, 1600, num_to_change], index = ['cancer_border', 'cancer_core', 'stroma', 'immune1'])

# for trial in range(5):
#     adata = qu.tl.simulate_structured_data(annotations_by_mask, cell_table, adata_expression, fov_key = 'fov', fov_list = fov_list, labels_key = 'mask_name', 
#                         cond1 = 'cancer_core', cond2 = 'cancer_border', radius = 500, p = 2, condition_id = 'immune1', num_to_change = num_to_change,
#                         compartment_colormap = compartment_colormap, prevalence = 0.6, ratio_df = ratio_df, cell_types = ['cancer_core', 'cancer_border', 'stroma', 'immune1'],
#                         sim_cell_types = ['Group1', 'Group1', 'Group2', 'Group3'], group_key = 'group', spatial_key = 'spatial', n_jobs = 8)
#     adata.write_h5ad(os.path.join('data', 'simulated', f'adata_simulated_structured_trial{trial}.h5ad'))

### define metrics for evaluation

In [ ]:
eval_method_list = [qu.tl.group_recall, qu.tl.evaluate_purity]
metric_names = ['recall', 'purity']
eval_method_params_list = [{'method_type':'other', 'feature_key':'spatial_nhood', 'ground_key': 'DA_group', 'labels_key': 'quiche_niche', 'ground_truth_niches':['border_immune1', 'core_immune1']},
                      {'annot_key':'quiche_niche', 'labels_key':'mask_name', 'fov_key':'Patient_ID', 'condition_key':'DA_group','feature_key':'spatial_nhood'}]

### number of nearest neighbors

In [ ]:
ksweep = [10, 20, 30, 40, 50, 75, 100]
spatial_method_params = {'radius': 200,
                         'labels_key':'mask_name',
                         'spatial_key':'spatial',
                         'fov_key':'Patient_ID',
                         'patient_key':'Patient_ID',
                         'delaunay': False,
                         'min_cells':3,
                         'k_sim':100,
                         'design':'~condition',
                         'model_contrasts':'conditioncancer_core-conditioncancer_border',
                         'sketch_size':None,
                         'nlargest': 4,
                         'annotation_key':'quiche_niche',
                         'n_jobs':-1,
                         'label_scheme':'normal'}

evaluation_df = pd.DataFrame()
for trial in range(5):
    adata = anndata.read_h5ad(os.path.join('data', 'simulated', f'adata_simulated_structured_trial{trial}.h5ad'))
    for n_neighbors in ksweep:
        for i in range(0, len(eval_method_list)):
            print(trial, n_neighbors, i)
            eval_method = eval_method_list[i]
            eval_method_params = eval_method_params_list[i]
            spatial_method_params['n_neighbors'] = n_neighbors
            mdata, _ = qu.tl.run_quiche(adata, **spatial_method_params)
            scores_df = pd.DataFrame(mdata['quiche'].var.groupby('quiche_niche')['PValue'].median())
            scores_df.columns = ['pval']
            scores_df['logFC'] = mdata['quiche'].var.groupby('quiche_niche')['logFC'].median()
            scores_df = scores_df[scores_df['pval'] < 0.05]
            ids = list(set(scores_df.index).intersection(set(list(mdata['quiche'].var['quiche_niche'].value_counts()[mdata['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
            scores_df = scores_df.loc[ids]
            eval_df = eval_method(mdata, scores_df = scores_df, **eval_method_params)
            eval_df.loc[:, 'metric'] = metric_names[i]
            eval_df.loc[:, 'n_neighbors'] = n_neighbors
            eval_df.loc[:, 'trial'] = trial
            evaluation_df = pd.concat([eval_df, evaluation_df], axis = 0)
evaluation_df.to_csv(os.path.join('data', 'output_files', 'param_n_neighbors.csv'))

### radius

In [ ]:
radius_list = [50, 75, 100, 200, 250, 300]
spatial_method_params = {'labels_key':'mask_name',
                         'spatial_key':'spatial',
                         'fov_key':'Patient_ID',
                         'n_neighbors': 30, 
                         'k_sim':100,
                         'patient_key':'Patient_ID',
                         'delaunay': False,
                         'min_cells':3,
                         'design':'~condition',
                         'model_contrasts':'conditioncancer_core-conditioncancer_border',
                         'sketch_size':None,
                         'nlargest': 4,
                         'annotation_key':'quiche_niche',
                         'n_jobs':-1,
                         'label_scheme':'normal'}

evaluation_df = pd.DataFrame()
for trial in range(5):
    adata = anndata.read_h5ad(os.path.join('data', 'simulated', f'adata_simulated_structured_trial{trial}.h5ad'))
    for radius in radius_list:
        for i in range(0, len(eval_method_list)):
            print(trial, radius, i)
            eval_method = eval_method_list[i]
            eval_method_params = eval_method_params_list[i]
            spatial_method_params['radius'] = radius
            mdata, _ = qu.tl.run_quiche(adata, **spatial_method_params)
            scores_df = pd.DataFrame(mdata['quiche'].var.groupby('quiche_niche')['PValue'].median())
            scores_df.columns = ['pval']
            scores_df['logFC'] = mdata['quiche'].var.groupby('quiche_niche')['logFC'].median()
            scores_df = scores_df[scores_df['pval'] < 0.05]
            ids = list(set(scores_df.index).intersection(set(list(mdata['quiche'].var['quiche_niche'].value_counts()[mdata['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
            scores_df = scores_df.loc[ids]
            eval_df = eval_method(mdata, scores_df = scores_df, **eval_method_params)
            eval_df.loc[:, 'metric'] = metric_names[i]
            eval_df.loc[:, 'radius'] = radius
            eval_df.loc[:, 'trial'] = trial
            evaluation_df = pd.concat([eval_df, evaluation_df], axis = 0)
evaluation_df.to_csv(os.path.join('data', 'output_files', 'param_radius.csv'))

### k sim

In [ ]:
ksweep = [50, 100, 200, 300]
spatial_method_params = {'labels_key':'mask_name',
                         'spatial_key':'spatial',
                         'fov_key':'Patient_ID',
                         'n_neighbors': 30, 
                         'radius': 200,
                         'patient_key':'Patient_ID',
                         'delaunay': False,
                         'min_cells':3,
                         'design':'~condition',
                         'model_contrasts':'conditioncancer_core-conditioncancer_border',
                         'sketch_size':None,
                         'nlargest': 4,
                         'annotation_key':'quiche_niche',
                         'n_jobs':-1,
                         'label_scheme':'normal'}

evaluation_df = pd.DataFrame()
for trial in range(5):
    adata = anndata.read_h5ad(os.path.join('data', 'simulated', f'adata_simulated_structured_trial{trial}.h5ad'))
    for k_sim in ksweep:
        for i in range(0, len(eval_method_list)):
            print(trial, k_sim, i)
            eval_method = eval_method_list[i]
            eval_method_params = eval_method_params_list[i]
            spatial_method_params['k_sim'] = k_sim
            mdata, _ = qu.tl.run_quiche(adata, **spatial_method_params)
            scores_df = pd.DataFrame(mdata['quiche'].var.groupby('quiche_niche')['PValue'].median())
            scores_df.columns = ['pval']
            scores_df['logFC'] = mdata['quiche'].var.groupby('quiche_niche')['logFC'].median()
            scores_df = scores_df[scores_df['pval'] < 0.05]
            ids = list(set(scores_df.index).intersection(set(list(mdata['quiche'].var['quiche_niche'].value_counts()[mdata['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
            scores_df = scores_df.loc[ids]
            eval_df = eval_method(mdata, scores_df = scores_df, **eval_method_params)
            eval_df.loc[:, 'metric'] = metric_names[i]
            eval_df.loc[:, 'k_sim'] = k_sim
            eval_df.loc[:, 'trial'] = trial
            evaluation_df = pd.concat([eval_df, evaluation_df], axis = 0)
evaluation_df.to_csv(os.path.join('data', 'output_files', 'param_k_sim.csv'))

### sketch size 

In [ ]:
sketch_size_list = [250, 500, 750, 1000, 2000, 2500, None]
spatial_method_params = {'labels_key':'mask_name',
                         'spatial_key':'spatial',
                         'fov_key':'Patient_ID',
                         'n_neighbors': 30, 
                         'radius': 200,
                         'patient_key':'Patient_ID',
                         'delaunay': False,
                         'min_cells':3,
                         'design':'~condition',
                         'k_sim': 100,
                         'model_contrasts':'conditioncancer_core-conditioncancer_border',
                         'nlargest': 4,
                         'annotation_key':'quiche_niche',
                         'n_jobs':-1,
                         'label_scheme':'normal'}

evaluation_df = pd.DataFrame()
for trial in range(5):
    adata = anndata.read_h5ad(os.path.join('data', 'simulated', f'adata_simulated_structured_trial{trial}.h5ad'))
    for sketch_size in sketch_size_list:
        for i in range(0, len(eval_method_list)):
            print(trial, sketch_size, i)
            eval_method = eval_method_list[i]
            eval_method_params = eval_method_params_list[i]
            spatial_method_params['sketch_size'] = sketch_size
            mdata, _ = qu.tl.run_quiche(adata, **spatial_method_params)
            scores_df = pd.DataFrame(mdata['quiche'].var.groupby('quiche_niche')['PValue'].median())
            scores_df.columns = ['pval']
            scores_df['logFC'] = mdata['quiche'].var.groupby('quiche_niche')['logFC'].median()
            scores_df = scores_df[scores_df['pval'] < 0.05]
            ids = list(set(scores_df.index).intersection(set(list(mdata['quiche'].var['quiche_niche'].value_counts()[mdata['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
            scores_df = scores_df.loc[ids]
            eval_df = eval_method(mdata, scores_df = scores_df, **eval_method_params)
            eval_df.loc[:, 'metric'] = metric_names[i]
            eval_df.loc[:, 'sketch'] = sketch_size
            eval_df.loc[:, 'trial'] = trial
            evaluation_df = pd.concat([eval_df, evaluation_df], axis = 0)
evaluation_df.to_csv(os.path.join('data', 'output_files', 'param_sketch.csv'))

In [ ]:
def plot_param(df, param_key, save_directory, filename_save):
    fig, axes = plt.subplots(2, 1, figsize=(3.5, 6.5), sharex=True)
    g = sns.lineplot(x = param_key, y = 'value', data = df[df['metric'] == 'recall'], color = 'k', markers=True, marker="o", ax = axes[0], ci= 'sd')
    g.tick_params(labelsize=8)
    g.set_xlabel('k', fontsize = 12)
    g.set_ylim(-0.05, 1.05)
    g.set_ylabel('niche recall', fontsize = 12)

    g = sns.lineplot(x = param_key, y = 'value', data = df[df['metric'] == 'purity'], color = 'k', markers=True, marker="o", ax = axes[1], ci= 'sd')
    g.tick_params(labelsize=8)
    g.set_xlabel(param_key, fontsize = 12)
    g.set_ylim(-0.05, 1.05)
    g.set_ylabel('niche purity', fontsize = 12)

    # Adjust layout and display the plot
    plt.tight_layout()
    plt.savefig(os.path.join(save_directory, filename_save+'.pdf'), bbox_inches = 'tight')

In [ ]:
save_directory = os.path.join('publications', 'supplementary_figures', 'supplementary_figure06')
qu.pp.make_directory(save_directory)

In [ ]:
for metric in ['n_neighbors', 'k_sim', 'radius', 'sketch']:
    df = pd.read_csv(os.path.join('data', 'output_files', f'param_{metric}.csv'), index_col = 0)
    if metric == 'sketch':
        df['sketch'][df['sketch'].isnull()] = 3650 #total cells per patient sample
        
    plot_param(df, metric, save_directory, f'param_{metric}')